In [41]:
import pandas as pd
import numpy as np
import os

In [42]:
os.chdir('/Users/arvind/Documents/Work/Zoho WorkDrive (Educational Initiatives Private Limited)/My Folders/Projects/SALT/Dashboards/')

In [43]:
df = pd.read_csv('./CBAScoredFiles/g6.csv')

In [47]:
drop = [
 'LQ16',
 'LQ17',
 'LQ18',
 'LQ19',
 'Score_LQ16',
 'Score_LQ17',
 'Score_LQ18',
 'Score_LQ19',
 'MQ16',
 'MQ17',
 'MQ18',
 'MQ19',
 'Score_MQ16',
 'Score_MQ17',
 'Score_MQ18',
 'Score_MQ19',
 'SQ16',
 'SQ17',
 'SQ18',
 'SQ19',
 'Score_SQ16',
 'Score_SQ17',
 'Score_SQ18',
 'Score_SQ19',
 'OQ16',
 'OQ17',
 'OQ18',
 'OQ19',
 'Score_OQ16',
 'Score_OQ17',
 'Score_OQ18',
 'Score_OQ19'
]

In [48]:
df.drop(drop, axis = 1, inplace=True)

In [49]:
unwanted = ('student_id','disecode','student_name','class','section','roll_no','unique_id','gender','social_category','medium_id'
            ,'Language_paperset','Language_jumble_set','Language_present_absent','LanguageTotal','LanguagePercentage_old','LanguagePercentage','Language_cheating_drop'
            ,'Maths_paperset','Maths_jumble_set','Maths_present_absent','MathsTotal','MathsPercentage_old','MathsPercentage','Maths_cheating_drop'
            ,'EVS_paperset','EVS_jumble_set','EVS_present_absent','EVSTotal','EVSPercentage_old','EVSPercentage','EVS_cheating_drop'
            ,'Science_paperset','Science_jumble_set','Science_present_absent','ScienceTotal','SciencePercentage_old','SciencePercentage','Science_cheating_drop'
            ,'SS_paperset','SS_jumble_set','SS_present_absent','SSTotal','SSPercentage_old','SSPercentage','SS_cheating_drop'
           )

In [50]:
var_list = list(df)

In [51]:
#EQ = EVS, MQ = Maths, LQ = Language (Telugu), SQ = Science, OQ = Social Science
prefixes = ('EQ','MQ','LQ','SQ','OQ')
dropped = [x for x in var_list if x.startswith(prefixes)]

In [52]:
df.drop(dropped, axis = 1 , inplace = True)

In [53]:
var_list = list(df)

In [54]:
retain = [ele for ele in var_list if ele not in unwanted]

In [55]:
#Define dimensions based on grade - present absent flag will change based on the subjects relevant to that grade
#G2 - Maths, Language; G3,4,5 - EVS, Maths, Language; G6,7,8 - Maths, Language, SS, Science
dimensions = ['unique_id','disecode','student_name','class'
                                  ,'gender','social_category'
                                  ,'Language_present_absent'
                                  ,'Maths_present_absent'
                                  ,'SS_present_absent'
                                  ,'Science_present_absent'
                                ]

In [56]:
df = pd.melt(df,id_vars = dimensions
                        ,value_vars = retain)

In [57]:
df[['S','Q']] = df['variable'].str.split('_',1,expand=True)

In [58]:
df[['Subject','Question']] = df['Q'].str.split('Q',1,expand=True)

In [59]:
#Language mapped as Telugu for general files; English and Hindi will be separate scored files altogether
df.loc[df["Subject"]=="L","Subject"]="Telugu"
df.loc[df["Subject"]=="M","Subject"]="Maths"
df.loc[df["Subject"]=="E","Subject"]="EVS"
df.loc[df["Subject"]=="O","Subject"]="SS"
df.loc[df["Subject"]=="S","Subject"]="Science"

In [60]:
df = df.drop(['variable','S','Q'],axis=1)

In [61]:
df.fillna('',inplace = True)

In [62]:
#Define dimensions based on grade - present absent flag will change based on the subjects relevant to that grade
#G2 - Maths, Language; G3,4,5 - EVS, Maths, Language; G6,7,8 - Maths, Language, SS, Science
indexELM = df[(((df['Language_present_absent']=='A')|(df['Language_present_absent']==''))&(df['Subject']=='Telugu'))|
                       (((df['Maths_present_absent']=='A')|(df['Maths_present_absent']==''))&(df['Subject']=='Maths'))|
                       (((df['SS_present_absent']=='A')|(df['SS_present_absent']==''))&(df['Subject']=='SS'))|
                       (((df['Science_present_absent']=='A')|(df['Science_present_absent']==''))&(df['Subject']=='Science'))
                      ].index

df.drop(indexELM, inplace=True)

In [63]:
#Define dimensions based on grade - present absent flag will change based on the subjects relevant to that grade
#G2 - Maths, Language; G3,4,5 - EVS, Maths, Language; G6,7,8 - Maths, Language, SS, Science
df = df.drop(['Language_present_absent','Maths_present_absent','SS_present_absent','Science_present_absent'],axis=1)

In [64]:
df['Question'] = df['Question'].astype(np.int64)

In [65]:
df = df.rename(columns = {'unique_id':'Child_Id','disecode':'Udise','student_name':'Child_Name'\
                          ,'class':'Grade','gender':'Gender','social_category':'Social_Category'\
                          ,'value':'Score','Subject':'Subject'\
                          ,'Question':'Question_no'})

<h1 style="color:blue;">
    School mapping
</h1>

In [66]:
school_mapping = pd.read_excel('./MappingFiles/AP_SchoolMaster_DistrictMandal_Mapping.xlsx')

In [67]:
# school_mapping['SCHOOLCODE'] = school_mapping['SCHOOLCODE'].astype(str)

In [68]:
school_mapping = school_mapping.rename(columns = {'SCHOOLCODE':'Udise'})

In [69]:
df = pd.merge(df,school_mapping,how='left',on='Udise')

In [70]:
question_mapping = pd.read_csv('./MappingFiles/cba1_answer_key_with_question_info.csv')

In [71]:
question_mapping['subject'] =  np.where(question_mapping['subject']=='Language'\
                                        , question_mapping['medium']\
                                        , question_mapping['subject'])

In [72]:
question_mapping['subject'] = np.where(question_mapping['subject']=='Biology','Science'\
                                       ,question_mapping['subject'])

In [73]:
question_mapping['subject'] = np.where(question_mapping['subject']=='Physics','Science'\
                                       ,question_mapping['subject'])

In [74]:
question_mapping = question_mapping[['class','subject','qno','skill_name','topic_name','characteristics']]

In [75]:
question_mapping.drop_duplicates(keep='first', inplace=True)

In [76]:
question_mapping = question_mapping.rename(columns={'class':'Grade','subject':'Subject','qno':'Question_no'})

In [77]:
df = pd.merge(df,question_mapping,how='left',on=['Grade','Subject','Question_no'])

In [78]:
df = df.assign(State='Andhra Pradesh')

In [79]:
df = df.rename(columns={'SCHOOLNAME':'School','schooltype':'School_Type','Udise':'Udise'\
                         ,'Social_Category':'Social_Category','District ':'District'\
                           ,'skill_name':'Skill','topic_name':'Topic','characteristics':'Characteristic'})

In [80]:
df = df[[
'State',
'District',
'Mandal',
'School',
'Udise',
'School_Type',
'Grade',
'Child_Name',
'Child_Id',
'Gender',
'Social_Category',
'Subject',
'Question_no',
'Score',
'Topic',
'Skill',
'Characteristic']]

In [81]:
df.to_pickle('./CBALongPickleFiles/G6.pkl')

In [82]:
#df.to_csv('./CBALongFiles/CBA_G5_LongFile.csv')

In [83]:
df = pd.DataFrame()